# Programa 1: Algoritmo ID3 a Mano
Para poder hacer el algoritmo ID3 utilizamos pandas para poder organizar los datos que vamos a categorizar de forma más sencilla y el modulo `train-test-selection`para crear un conjunto de datos de prueba y otro de entrenamiento.

In [8]:
import pandas as pd
import pprint
from sklearn.model_selection import train_test_split
from math import log


## Carga de datos
Cargamos los datos que habiamos guardado previamente en un CSV a un dataframe de pandas.

In [9]:
df1 = pd.DataFrame.from_csv("./data/Ejemplo12.csv", header=0)
df1

,Precion arterial,Azucar en la Sangre,Colesterol,Alergia Antibiotico,Otras Alergias,Administrar Farmaco
1,Alta,Alto,Alto,No,No,Si
2,Alta,Alto,Alto,Si,No,Si
3,Baja,Alto,Bajo,No,No,Si
4,Media,Alto,Alto,No,Si,No
5,Media,Bajo,Alto,Si,Si,No
6,Baja,Bajo,Alto,Si,Si,Si
7,Alta,Bajo,Alto,Si,No,Si
8,Alta,Bajo,Bajo,No,Si,Si
9,Alta,Alto,Bajo,Si,Si,No
10,Baja,Bajo,Alto,Si,Si,Si


## Definicion de Funciones
En esta seccion definimos algunas funciones y clases que estaremos utilizando para calcular el arbol. En este caso, utilizamos las funciones `entropy` y `calcInitEntropy` para obtener los valores de la entropia de los valores que tenemos en el dataframe. Despues tenemos la clase `ID3_Classifier` que es nuestro clasificador ID3. Esta clase tiene dos funciones hablitiadas hasta el momento:
1. train -> Entrena el algoritmo
2. printTree -> Imprime el arbol.

In [10]:
def entropy(p,k):
    '''
    Calcula el valor de la entropia de los valores k y p.
    
    Args
    ----
    p -- entero positivo
    k -- entero positivo
    
    Return
    -------
    entropy -- entero positivo con valor de la entropia
    '''
    try:
        return p/(p+k)*log(p/(p+k),2) + k/(p+k)*log(k/(p+k),2)
    except ValueError:
        return 0
    
def calcInitEntropy(df):
    '''
    Calcula la entropia inicial del conjunto de datos
    
    Arg
    ---
    df -- Pandas DataFrame
    
    Returns
    --------
    entropy -- Valor de la entropia inicial (float)
    '''
    p = df.groupby(df.columns[-1],as_index=False).size()[1]
    k = df.groupby(df.columns[-1],as_index=False).size()[0]
    return abs(entropy(p,k))


In [11]:
class ID3_Classifier():
    '''Clasificador de tipo arbol de decision usando entropias como criterio (ID3)'''
    def __init__(self):
        modelo = {}

    def train(self, dataframe):
        '''
        Genera un arbol utilizando entropias como caracteristica principal.

        Args
        -----
        dataframe -- Pandas Dataframe con datos. La ultima columna debe contener los targets.

        Returns
        --------
        arbol -- Diccionario con los nodos

        '''
        #Guardamos los resultados en un diccionario llamado arbol
        arbol = {}
        frame = dataframe
        # Arreglo que almacena las entropias del conjunto de caracteristicas actuales.
        entropias = []
        # Indice del elemento con mayor ganancia (preinicializado en 0)
        max_index = 0

        # Calculamos la entropia inicial
        entropia_inicial = calcInitEntropy(frame)

        # Entramos a cada una de las columnas para ver cual tiene la mayor ganancia.
        for i in range(len(frame.columns) - 1):
            entropias.append(entropia_inicial)
            tags = []

            # Agrupamos por caracteristica y por si el resultado es Si o No y lo almacenamos en df_temp
            df_temp = frame.groupby([frame.columns[i], frame.columns[-1]], as_index=False).size()

            #Por cada caracteristica buscamos su valor Si o No
            for tag in frame[frame.columns[i]].unique():
                # print(tag)
                try:
                    p = df_temp[tag]['No']
                except KeyError:
                    # Si no existe valor No entonces atrapa la excepcion y le asigna un valor de 0
                    p = 0
                try:
                    k = df_temp[tag]['Si']
                except KeyError:
                    # Si no existe valor Si entonces atrapa la excepcion y le asigna un valor de 0
                    k = 0

                entropias[i] += ((p+k)/df_temp.sum())*entropy(p,k)
                max_index = entropias.index(max(entropias))

        # Obtenemos el indice de la columna con la mayor entropia
        arbol[frame.columns[entropias.index(max(entropias))]] = {}
        df_temp = frame.groupby([frame.columns[max_index], frame.columns[-1]], as_index=False).size()

        # Verificamos cuales caracteristicas derivan en sub nodos u hojas finales
        for tag in frame[frame.columns[max_index]].unique():
            try:
                p = df_temp[tag]['No']
            except KeyError:
                # Si no existe valor No entonces atrapa la excepcion y le asigna un valor de 0
                p = 0
            try:
                k = df_temp[tag]['Si']
            except KeyError:
                # Si no existe valor Si entonces atrapa la excepcion y le asigna un valor de 0
                k = 0
            if p > 0 and k > 0:
                # Tiene un valor de Entropia. Creamos un subframe con los datos del tag unicamente.
                subframe = frame.loc[frame[frame.columns[max_index]] == tag]
                
                # Eliminamos la columna que estamos analizando para evitar que aparezca dos veces una
                # misma caracteristica en la misma rama.
                del subframe[subframe.columns[max_index]]

                arbol[frame.columns[max_index]][tag] = {}
                try:
                    arbol[frame.columns[max_index]][tag].update(self.train(subframe))
                except ValueError:
                    arbol[frame.columns[max_index]][tag] = "?"

            elif k > 0:
                # K es mayor que 0 Solo tiene Si
                arbol[frame.columns[entropias.index(max(entropias))]][tag] = '+'

            elif p > 0:
                # P es mayor que 0. Solo tiene No
                arbol[frame.columns[entropias.index(max(entropias))]][tag] = '-'

            else:
                # Los dos son 0. Por lo tanto, es ?
                arbol[frame.columns[entropias.index(max(entropias))]][tag] = '?'
            
        self.modelo = arbol
        return arbol
    
    def printTree(self):
        '''Imprime el arbol de decision'''
        pprint.pprint(self.modelo, width = 2)

In [12]:
clasificador = ID3_Classifier()
clasificador.train(df1)
clasificador.printTree()

{'Precion arterial': {'Alta': {'Otras Alergias': {'No': '+',
                                                  'Si': {'Alergia Antibiotico': {'No': '+',
                                                                                 'Si': '-'}}}},
                      'Baja': '+',
                      'Media': {'Colesterol': {'Alto': '-',
                                               'Bajo': '+'}}}}
